In [6]:
# 모델 로드
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os
import random
from PIL import Image
import numpy as np

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from pathlib import Path

# 🔹 업로드한 폰트 경로 지정
font_path = Path("/workspace/fonts/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothic.ttf")

if font_path.exists():
    fm.fontManager.addfont(str(font_path))
    font_name = fm.FontProperties(fname=str(font_path)).get_name()
    plt.rc("font", family=font_name)
    plt.rcParams["axes.unicode_minus"] = False
    print(f"✅ 나눔고딕 폰트 등록 완료! ({font_name})")
else:
    print("❌ 폰트를 찾을 수 없습니다. 경로를 확인하세요:", font_path)

In [7]:
# CBAM block 정의 define
# CBAM 은 Channel attention module + Spatial attention module

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                                nn.ReLU(),
                                nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_out = torch.cat([avg_out, max_out], dim=1)
        x_out = self.conv1(x_out)
        return self.sigmoid(x_out)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)  # Channel Attention
        out = out * self.sa(out) # Spatial Attention
        return out

## CBAM 블록 정의 끝

In [8]:
# CBAM 통합 CNN 모델 define
class BaselineCBAMModel(nn.Module):
    def __init__(self, num_classes=2):
        super(BaselineCBAMModel, self).__init__()
        
        # 첫 번째 CNN 블록
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2) # 2배 축소
        )
        self.cbam1 = CBAM(16)

        # 두 번째 CNN 블록
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2) # 2배 축소
        )
        self.cbam2 = CBAM(32)
        
        # 세 번째 CNN 블록
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2) # 2배 축소
        )
        # CBAM은 마지막 피쳐 맵 전에 추가할 수 있습니다. (선택적)
        
        # Fully Connected Layer (입력 크기는 이미지 크기에 따라 달라짐)
        # 예시: 224x224 -> 28x28. 64 * 28 * 28을 가정.
        # 실제 입력 이미지 크기에 맞춰 `self._get_conv_output`을 사용하거나 추론 필요
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # Global Average Pooling
        self.fc = nn.Linear(64, num_classes) 

    def forward(self, x):
        x = self.layer1(x)
        x = self.cbam1(x)
        
        x = self.layer2(x)
        x = self.cbam2(x)

        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1) # Flatten (Batch_size, 64)
        x = self.fc(x)
        return x

In [9]:
## 시드 고정
import torch
import numpy as np
import random
import os

def set_reproducible_seed(seed_value):
    """
    모든 난수 생성기의 시드를 고정하여 재현성을 보장합니다.
    """
    # 1. Python random seed 고정
    random.seed(seed_value)
    
    # 2. NumPy seed 고정
    np.random.seed(seed_value)
    
    # 3. PyTorch CPU seed 고정
    torch.manual_seed(seed_value)
    
    # 4. PyTorch CUDA (GPU) seed 고정 (GPU 사용 시)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # 멀티 GPU 환경일 경우
        
        # 5. CUDA 관련 설정 (결정적 알고리즘 사용)
        # 이 설정은 성능 저하를 초래할 수 있지만, 완벽한 재현성을 위해 필요합니다.
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
    # 6. 환경 변수 설정 (필요에 따라)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [10]:
## 커스텀 데이터셋 및 데이터 로더
class EmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        
        # 이진 분류 레이블 맵핑
        self.label_map = {
            'happy': 1,
            'anger': 0,
            'sadness': 0,
            'panic': 0
        }
        
        # 이미지 경로 및 레이블 수집
        for emotion_folder in os.listdir(root_dir):
            emotion_path = os.path.join(root_dir, emotion_folder)
            if os.path.isdir(emotion_path) and emotion_folder in self.label_map:
                label = self.label_map[emotion_folder]
                for image_file in os.listdir(emotion_path):
                    if image_file.endswith(('.jpg', '.png', '.jpeg')):
                        self.images.append((os.path.join(emotion_path, image_file), label))
        
        random.shuffle(self.images) # Train/Val 나뉘어 있으므로 섞지 않아도 되지만, 혹시 모를 편향 방지

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [ ]:
## 학습 실행 코드

# --- 환경 설정 ---
ROOT_DATA_DIR = '/workspace/merge_data/data/img/' # 실제 데이터셋 루트 경로로 변경하세요.
IMG_SIZE = 64 # 원본 데이터 크기 그대로 사용합니다. (예: 64x64)
BATCH_SIZE = 32
NUM_EPOCHS = 10
SEED = 42
MODEL_NAME = '/workspace/user1/baseline_v1.pth'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 데이터 전처리 (Crop, Augmentation 금지) ---
# 원본 데이터 그대로 사용하므로 크기 조정(Resize)과 Tensor 변환만 수행합니다.
data_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(), # [0, 1]로 정규화
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 정규화는 선택 사항이지만, 베이스라인에서는 제외할 수도 있습니다. 포함하여 진행합니다.
])

# --- 데이터셋 및 데이터로더 로드 ---
train_dataset = EmotionDataset(root_dir=os.path.join(ROOT_DATA_DIR, 'train'), transform=data_transform)
val_dataset = EmotionDataset(root_dir=os.path.join(ROOT_DATA_DIR, 'val'), transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 모델, 손실 함수, 옵티마이저 초기화 ---
model = BaselineCBAMModel(num_classes=2).to(DEVICE)
# 이진 분류이므로 CrossEntropyLoss 또는 BCEWithLogitsLoss를 사용합니다.
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 학습 함수 ---
def train_model():
    best_val_acc = 0.0
    
    for epoch in range(NUM_EPOCHS):
        # TRAIN
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss_train = running_loss / len(train_dataset)
        epoch_acc_train = correct_train / total_train

        # VALIDATION
        model.eval()
        running_loss_val = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                running_loss_val += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        epoch_loss_val = running_loss_val / len(val_dataset)
        epoch_acc_val = correct_val / total_val
        
        print(f'Epoch {epoch+1}/{NUM_EPOCHS}:')
        print(f'  Train Loss: {epoch_loss_train:.4f}, Train Acc: {epoch_acc_train:.4f}')
        print(f'  Val Loss: {epoch_loss_val:.4f}, Val Acc: {epoch_acc_val:.4f}')
        
        # 모델 가중치 저장 (최종 에폭 또는 최고 성능 모델)
        if epoch + 1 == NUM_EPOCHS:
            print(f"\n✅ 최종 에폭 모델 가중치 저장: {MODEL_NAME}")
            torch.save(model.state_dict(), MODEL_NAME)

# --- 학습 실행 ---
if __name__ == '__main__':
    # ⭐ 시드 설정 함수 호출 위치 (가장 중요)
    set_reproducible_seed(SEED)

    # 데이터 경로가 올바른지 확인하는 로직 추가 필요
    if not os.path.exists(os.path.join(ROOT_DATA_DIR, 'train')) or not os.path.exists(os.path.join(ROOT_DATA_DIR, 'val')):
        print("❌ 오류: ROOT_DATA_DIR 설정을 확인하거나, 데이터 폴더(train/val) 구조가 올바른지 확인하세요.")
    else:
        train_model() # 여기서 모델 초기화 및 학습이 시작됩니다.

Epoch 1/10:
  Train Loss: 0.5753, Train Acc: 0.7377
  Val Loss: 0.5779, Val Acc: 0.7402
Epoch 2/10:
  Train Loss: 0.5689, Train Acc: 0.7444
  Val Loss: 0.5825, Val Acc: 0.7402
Epoch 3/10:
  Train Loss: 0.5684, Train Acc: 0.7444
  Val Loss: 0.5731, Val Acc: 0.7402
Epoch 4/10:
  Train Loss: 0.5667, Train Acc: 0.7444
  Val Loss: 0.5846, Val Acc: 0.7402
Epoch 5/10:
  Train Loss: 0.5667, Train Acc: 0.7444
  Val Loss: 0.5818, Val Acc: 0.7402
Epoch 6/10:
  Train Loss: 0.5631, Train Acc: 0.7442
  Val Loss: 0.5776, Val Acc: 0.7402
Epoch 7/10:
  Train Loss: 0.5553, Train Acc: 0.7444
  Val Loss: 0.5761, Val Acc: 0.7428
Epoch 8/10:
  Train Loss: 0.5444, Train Acc: 0.7442
  Val Loss: 0.5570, Val Acc: 0.7333
Epoch 9/10:
  Train Loss: 0.5322, Train Acc: 0.7471
  Val Loss: 0.7474, Val Acc: 0.7402
Epoch 10/10:
  Train Loss: 0.5178, Train Acc: 0.7550
  Val Loss: 0.5967, Val Acc: 0.7411

✅ 최종 에폭 모델 가중치 저장: baseline_v1.pth


: 